# Train a Hypergraph Neural Network

In this notebook, we will create and train a two-step message passing network in the hypergraph domain. We will use a benchmark dataset, shrec16, a collection of 3D meshes, to train the model to perform classification at the level of the hypergraph. 

In [1]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split

from toponetx import SimplicialComplex
import toponetx.datasets as datasets
from topomodelx.nn.hypergraph.allset_layer import AllSetLayer

from torch_geometric.utils import to_edge_index
# make ipynb to read .py files continiously
%load_ext autoreload
%autoreload 2



/Users/leone/anaconda3/envs/topo/lib/python3.11/site-packages/torch_geometric/typing.py:33: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: dlopen(/Users/leone/anaconda3/envs/topo/lib/python3.11/site-packages/torch_scatter/_version_cpu.so, 0x0006): tried: '/Users/leone/anaconda3/envs/topo/lib/python3.11/site-packages/torch_scatter/_version_cpu.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/Users/leone/anaconda3/envs/topo/lib/python3.11/site-packages/torch_geometric/typing.py:64: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: dlopen(/Users/leone/anaconda3/envs/topo/lib/python3.11/site-packages/torch_sparse/_version_cpu.so, 0x0006): tried: '/Users/leone/anaconda3/envs/topo/lib/python3.11/site-packages/torch_sparse/_version_cpu.so' (mach-o file, but is an incompatible architecture (h

If GPU's are available, we will make use of them. Otherwise, this will run on CPU.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Pre-processing

## Import data ##

The first step is to import the dataset, shrec 16, a benchmark dataset for 3D mesh classification. We then lift each graph into our domain of choice, a hypergraph.

We will also retrieve:
- input signal on the edges for each of these hypergraphs, as that will be what we feed the model in input
- the label associated to the hypergraph

In [3]:
shrec, _ = datasets.mesh.shrec_16(size="small")

shrec = {key: np.array(value) for key, value in shrec.items()}
x_0s = shrec["node_feat"]
x_1s = shrec["edge_feat"]
x_2s = shrec["face_feat"]

ys = shrec["label"]
simplexes = shrec["complexes"]

Loading dataset...

done!


In [4]:
i_complex = 6
print(
    f"The {i_complex}th simplicial complex has {x_0s[i_complex].shape[0]} nodes with features of dimension {x_0s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_1s[i_complex].shape[0]} edges with features of dimension {x_1s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_2s[i_complex].shape[0]} faces with features of dimension {x_2s[i_complex].shape[1]}."
)

The 6th simplicial complex has 252 nodes with features of dimension 6.
The 6th simplicial complex has 750 edges with features of dimension 10.
The 6th simplicial complex has 500 faces with features of dimension 7.


## Define neighborhood structures and lift into hypergraph domain. ##

Now we retrieve the neighborhood structures (i.e. their representative matrices) that we will use to send messges on each simplicial complex. In the case of this architecture, we need the boundary matrix (or incidence matrix) $B_1$ with shape $n_\text{nodes} \times n_\text{edges}$.

Once we have recorded the incidence matrix (note that all incidence amtrices in the hypergraph domain must be unsigned), we lift each simplicial complex into a hypergraph. The pairwise edges will become pairwise hyperedges, and faces in the simplciial complex will become 3-wise hyperedges.

In [5]:
hg_list = []
incidence_1_list = []
for simplex in simplexes:
    incidence_1 = simplex.incidence_matrix(rank=1, signed=False)
    # incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse()
    # incidence_1_list.append(incidence_1)
    hg = simplex.to_hypergraph()
    hg_list.append(hg)


# Extract hypergraphs incident matrices from collected hypergraphs
for hg in hg_list:
    incidence_1 = hg.incidence_matrix()
    incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse_csr()
    incidence_1_list.append(incidence_1)

/var/folders/gm/kdqr0d7n3y1cyygtb_y421tc0000gn/T/ipykernel_18771/2785629712.py:14: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:56.)
  incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse_csr()


In [6]:
i_complex = 6
print(
    f"The {i_complex}th hypergraph has an incidence matrix of shape {incidence_1_list[i_complex].shape}."
)

The 6th hypergraph has an incidence matrix of shape torch.Size([252, 1250]).


# Create the Neural Network

Using the TemplateLayer class, we create a neural network with stacked layers.

In [7]:
channels_edge = x_1s[0].shape[1]
channels_node = x_0s[0].shape[1]

In [8]:
class AllSetNN(torch.nn.Module):
    """AllSet Neural Network Module.

    A module that combines multiple AllSet layers to form a neural network.

    Parameters
    ----------
    in_dim : int
        Dimension of the input features.
    hid_dim : int
        Dimension of the hidden features.
    out_dim : int
        Dimension of the output features.
    dropout : float
        Dropout probability.
    n_layers : int, optional
        Number of AllSet layers in the network. Defaults to 2.
    input_dropout : float, optional
        Dropout probability for the layer input. Defaults to 0.2.
    mlp_num_layers : int, optional
        Number of layers in the MLP. Defaults to 2.
    mlp_input_norm : bool, optional
        Whether to apply input normalization in the MLP. Defaults to False.
    """

    def __init__(self, in_dim, hid_dim, out_dim,
                 n_layers=2, dropout=0.2, input_dropout=0.2,
                 mlp_num_layers=2, mlp_input_norm=False):
        super().__init__()
        layers = [
            AllSetLayer(
                in_dim=in_dim,
                hid_dim=hid_dim,
                out_dim=hid_dim,
                dropout=dropout,
                input_dropout=input_dropout,
                mlp_num_layers=mlp_num_layers,
                mlp_input_norm=mlp_input_norm
            )
        ]

        for _ in range(n_layers - 1):
            layers.append(
                AllSetLayer(
                    in_dim=hid_dim,
                    hid_dim=hid_dim,
                    out_dim=hid_dim,
                    dropout=dropout,
                    input_dropout=input_dropout,
                    mlp_num_layers=mlp_num_layers,
                    mlp_input_norm=mlp_input_norm
                )
            )
        self.layers = torch.nn.ModuleList(layers)
        self.linear = torch.nn.Linear(hid_dim, out_dim)

    def forward(self, x_0, incidence_1):
        """
        Forward computation.

        Parameters
        ----------
        x : torch.Tensor
            Input features.
        edge_index : torch.Tensor
            Edge list (of size (2, |E|)).

        Returns
        -------
        torch.Tensor
            Output prediction.
        """
        # cidx = edge_index[1].min()
        # edge_index[1] -= cidx
        # reversed_edge_index = torch.stack(
        #     [edge_index[1], edge_index[0]], dim=0)

        for layer in self.layers:
            x_0 = layer(x_0, incidence_1)
        pooled_x = torch.max(x_0, dim=0)[0]
        return torch.sigmoid(self.linear(pooled_x))[0]



# Train the Neural Network

We specify the model, the loss, and an optimizer.

In [31]:
#model = AllSetNN(channels_edge, channels_node, n_layers=2)
# ADD edge channels as well, it seems to be necessary for hypergraph NNs
hid_dim, out_dim = 64, 1
model = AllSetNN(in_dim=channels_node, hid_dim=hid_dim, out_dim=out_dim, n_layers=1)
model = model.to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.1)

Split the dataset into train and test sets.

In [32]:
test_size = 0.2
x_0_train, x_0_test = train_test_split(x_0s, test_size=test_size, shuffle=False)
incidence_1_train, incidence_1_test = train_test_split(
    incidence_1_list, test_size=test_size, shuffle=False
)
y_train, y_test = train_test_split(ys, test_size=test_size, shuffle=False)

The following cell performs the training, looping over the network for a low amount of epochs. We keep training minimal for the purpose of rapid testing.

In [33]:
loss_fn = torch.nn.MSELoss()

In [34]:
test_interval = 10
num_epochs = 50
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    for x_0, incidence_1, y in zip(x_0_train, incidence_1_train, y_train):
        x_0= torch.tensor(x_0)
        x_0, incidence_1, y = (
            x_0.float().to(device),
            incidence_1.float().to(device),
            torch.tensor(y, dtype=torch.float).to(device),
        )
        opt.zero_grad()
        # Extract edge_index from sparse incidence matrix
        #edge_index, _ = to_edge_index(incidence_1)
        y_hat = model(x_0, incidence_1)
        loss = loss_fn(y_hat, y)

        loss.backward()
        opt.step()
        epoch_loss.append(loss.item())

    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            for x_0, incidence_1, y in zip(x_0_test, incidence_1_test, y_test):
                x_0 = torch.tensor(x_0)
                x_0, incidence_1, y = (
                    x_0.float().to(device),
                    incidence_1.float().to(device),
                    torch.tensor(y, dtype=torch.float).to(device),
                )
                y_hat = model(x_0, incidence_1)
                loss = loss_fn(y_hat, y)

            print(f"Test_loss: {loss:.4f}", flush=True)

Epoch: 1 loss: 274.9095
Epoch: 2 loss: 274.6125
Epoch: 3 loss: 274.6125
Epoch: 4 loss: 274.6125
Epoch: 5 loss: 274.6125
Epoch: 6 loss: 274.6125
Epoch: 7 loss: 274.6125
Epoch: 8 loss: 274.6125
Epoch: 9 loss: 274.6125
Epoch: 10 loss: 274.6125
Test_loss: 529.0000
Epoch: 11 loss: 274.6125
Epoch: 12 loss: 274.6125
Epoch: 13 loss: 274.6125
Epoch: 14 loss: 274.6125
Epoch: 15 loss: 274.6125
Epoch: 16 loss: 274.6125
Epoch: 17 loss: 274.6125
Epoch: 18 loss: 274.6125
Epoch: 19 loss: 274.6125
Epoch: 20 loss: 274.6125
Test_loss: 529.0000
Epoch: 21 loss: 274.6125
Epoch: 22 loss: 274.6125
Epoch: 23 loss: 274.6125
Epoch: 24 loss: 274.6125
Epoch: 25 loss: 274.6125


KeyboardInterrupt: 